# ***Importing features and reading in data***

In [ ]:
!pip install rdkit-pypi --quiet
#!pip install mordred --quiet
!pip install lazypredict --quiet
!pip install sklearn-json --quiet
!pip install shap --quiet
!pip install optuna --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.4/29.4 MB 66.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.9/547.9 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 404.2/404.2 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.0/226.0 kB 23.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 kB 9.0 MB/s eta 0:00:00


In [ ]:
from rdkit.Chem import AllChem, Descriptors, PandasTools, rdMolDescriptors, Draw
from rdkit import Chem
from rdkit.ML.Descriptors import MoleculeDescriptors
from tqdm import tqdm
#--------------------------------------------
import pandas as pd
import numpy as np
import scipy
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import seaborn as sns
import shap
#--------------------------------------------
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from sklearn.model_selection import ShuffleSplit, cross_validate, train_test_split, KFold, cross_val_score, learning_curve
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.inspection import permutation_importance
from lazypredict.Supervised import LazyRegressor
from lightgbm import LGBMRegressor
#--------------------------------------------
import optuna

sns.set_theme(style="ticks")

%matplotlib inline

import warnings
import time

warnings.filterwarnings('ignore')
warnings.filterwarnings(action='ignore',category=DeprecationWarning)
warnings.filterwarnings(action='ignore',category=FutureWarning)

Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)


In [ ]:
seed =12

# Defining useful functions

def rmse(predictions, targets):
    return np.sqrt(((predictions - targets)**2).mean())

def test_learner(X, y, model):
    kf = KFold(n_splits=5, random_state=seed, shuffle=True)
    #X = X.values
    kf.get_n_splits(X)

    errorlist = []
    train_err = []
    corr_vals = []
    train_corr = []
    for train_idx, test_idx in kf.split(X):
        X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
        y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]

        model.fit(X_train, y_train)

        error = rmse(model.predict(X_test), y_test)
        training_error = rmse(model.predict(X_train), y_train)

        _, _, r_value, _, _ = scipy.stats.linregress(y_train, model.predict(X_train))
        r2_train = r_value**2

        _, _, r_value, _, _ = scipy.stats.linregress(y_test, model.predict(X_test))
        r2_test = r_value**2

        errorlist.append(error)
        train_err.append(training_error)

        corr_vals.append(r2_test)
        train_corr.append(r2_train)

    print(f'Final Train RMSE: {np.mean(train_err)} +/- {np.std(train_err)}')
    print(f'Final Train R^2: {np.mean(train_corr)} +/- {np.std(train_corr)}')
    print(f'Final Val RMSE: {np.mean(errorlist)} +/- {np.std(errorlist)}')
    print(f'Final Val R^2: {np.mean(corr_vals)} +/- {np.std(corr_vals)}')

def test_learner_np(X, y, model):
    kf = KFold(n_splits=5, random_state=seed, shuffle=True)
    X = X.values
    kf.get_n_splits(X)

    errorlist = []
    train_err = []
    corr_vals = []
    train_corr = []
    for train_idx, test_idx in kf.split(X):
        X_train, X_test = X[train_idx], X[test_idx]
        y_train, y_test = y[train_idx], y[test_idx]

        model.fit(X_train, y_train)

        error = rmse(model.predict(X_test), y_test)
        training_error = rmse(model.predict(X_train), y_train)

        _, _, r_value, _, _ = scipy.stats.linregress(y_train, model.predict(X_train))
        r2_train = r_value**2

        _, _, r_value, _, _ = scipy.stats.linregress(y_test, model.predict(X_test))
        r2_test = r_value**2

        errorlist.append(error)
        train_err.append(training_error)

        corr_vals.append(r2_test)
        train_corr.append(r2_train)

    print(f'Final Train RMSE: {np.mean(train_err)} +/- {np.std(train_err)}')
    print(f'Final Train R^2: {np.mean(train_corr)} +/- {np.std(train_corr)}')
    print(f'Final Val RMSE: {np.mean(errorlist)} +/- {np.std(errorlist)}')
    print(f'Final Val R^2: {np.mean(corr_vals)} +/- {np.std(corr_vals)}')

def canonical_smiles(smiles):
  mols = [Chem.MolFromSmiles(smi) for smi in smiles]
  smiles = [Chem.MolToSmiles(mol) for mol in mols]
  return smiles

def RDkit_descriptors(smiles):
  mols = [Chem.MolFromSmiles(i) for i in smiles]
  calc = MoleculeDescriptors.MolecularDescriptorCalculator([x[0] for x in Descriptors._descList])
  desc_names = calc.GetDescriptorNames()

  Mol_descriptors = list()
  for mol in mols:
    mol = Chem.AddHs(mol)
    descriptors = calc.CalcDescriptors(mol)
    Mol_descriptors.append(descriptors)
  return Mol_descriptors,desc_names

def remove_correlated_features(descriptors):
    # Calculate correlation
    correlated_matrix = descriptors.corr().abs()

    # Upper triangle of correlation matrix
    upper_triangle = correlated_matrix.where(np.triu(np.ones(correlated_matrix.shape),k=1).astype(np.bool))

    # Identify columns that have above 0.9 values of correlation
    to_drop = [column for column in upper_triangle.columns if any(upper_triangle[column] >= 0.9)]
    print(to_drop)
    descriptors_correlated_dropped = descriptors.drop(columns=to_drop, axis=1)
    return descriptors_correlated_dropped

def plotter(model, X_train, y_train, X_test, y_test):
  fig, ax = plt.subplots()
  ax.scatter(y_train, model.predict(np.array(X_train)), color="#008080")
  ax.scatter(y_test, model.predict(np.array(X_test)), color="#FF7F50")

  lims = [
      np.min([ax.get_xlim(), ax.get_ylim()]),  # min of both axes
      np.max([ax.get_xlim(), ax.get_ylim()]),  # max of both axes
  ]

  _, _, r_value, _, _ = scipy.stats.linregress(y_train, model.predict(X_train))
  r2_train = r_value**2

  _, _, r_value, _, _ = scipy.stats.linregress(y_test, model.predict(X_test))
  r2_test = r_value**2

  plt.annotate(f"Training R$^2$ = {r2_train:.3f}", (2050, 2110))
  plt.annotate(f"Testing R$^2$ = {r2_test:.3f}", (2050, 2105))

  # now plot both limits against eachother
  ax.plot(lims, lims, 'k-', alpha=0.75, zorder=0)
  ax.set_aspect('equal')
  ax.set_xlim(lims)
  ax.set_ylim(lims)
  ax.set_xlabel("DFT TEP (cm$^{-1}$)")
  ax.set_ylabel("Predicted TEP (cm$^{-1}$)")

  print(f"Training MAE: {mean_absolute_error(y_train, model.predict(X_train)):.02f} cm^-1")
  print(f"Testing MAE: {mean_absolute_error(y_test, model.predict(X_test)):.02f} cm^-1")
  print(f"Training RMSE: {rmse(y_train, model.predict(X_train)):.02f} cm^-1")
  print(f"Testing RMSE: {rmse(y_test, model.predict(X_test)):.02f} cm^-1")
  print(f"MAE / Range(test) = {(mean_absolute_error(y_test, model.predict(X_test)) *100 / (max(y_train)-min(y_train))):.02f}%")
  print(f"RMSE / Range(test) = {(rmse(y_test, model.predict(X_test)) *100 / (max(y_train)-min(y_train))):.02f}%")
  plt.show()

In [ ]:
# Reading in the CSV with the quantum chemical descriptors

df = pd.read_excel('_20230712-TEPid.xlsx', sheet_name='DATA')

y = df['TEPcorr']

functionalizations = df.iloc[:,3:].values

alphabetical = list()

for i in functionalizations:
  sortedlist = sorted(i)
  alphabetical.append(sortedlist)

#alphabetical = sorted(alphabetical)

unique = list()
unique_df = pd.DataFrame()

for i in alphabetical:
  if i in unique:
    pass
  else:
    unique.append(i)
    # unique_df.loc[i] = df.loc[i]

functionalizations = pd.DataFrame(unique, columns=['R1', 'R2', 'R3'])

functionalizations = functionalizations.sort_values(['R1', 'R2', 'R3'], ascending=(False, False, False))
functionalizations.head()

FileNotFoundError: ignored

In [ ]:
funcs_df = pd.DataFrame(alphabetical, columns=['R1','R2','R3'])

df['R1'] = funcs_df['R1']
df['R2'] = funcs_df['R2']
df['R3'] = funcs_df['R3']

df

NameError: ignored

In [ ]:
unique_df = pd.DataFrame(unique, columns=['R1','R2','R3'])

updated_df = unique_df.merge(df, on=['R1', 'R2', 'R3'], how='left').drop_duplicates(['R1', 'R2', 'R3'])
updated_df = updated_df.reset_index()
del updated_df['index']
updated_df

In [ ]:
updated_df.to_csv('UPDATED_TEPid_20230715.csv')

In [ ]:
sns.histplot(data = updated_df['TEPcorr'], kde = True)
plt.xlabel('DFT Corrected Tolman Electronic Parameter (cm$^{-1}$)')
#plt.savefig('BARRIER_kde.png', dpi=2500)
plt.show()

# Making SMILES strings

In [ ]:
R1 = updated_df.iloc[:,0]
R2 = updated_df.iloc[:,1]
R3 = updated_df.iloc[:,2]

ligand_dict_R1 = {
    '26-F-Ph': 'C3=C(F)C=CC=C3F',
    '35-F-Ph': 'C1=CC(F)=CC(F)=C1',
    'Br': 'Br',
    'CF3': 'C(F)(F)F',
    'CH2Cl': 'C(Cl)',
    'CH2F': 'C(F)',
    'CH2OH': 'CO',
    'CHO': 'C=O',
    'Cl': 'Cl',
    'COCH3': 'C(C)=O',
    'CONH2': 'C(N)=O',
    'COOCH3': 'C(OC)=O',
    'COOH': 'C(O)=O',
    'CN': 'C#N',
    'Cy': 'C1CCCCC1',
    'Et': 'CC',
    'F': 'F',
    'H': '',
    'I': 'I',
    'iPr': 'C(C)C',
    'Me': 'C',
    'Mes': 'C1=C(C)C=C(C)C=C1(C)',
    'NCH3_2': 'N(C)C',
    'NHCH3': 'NC',
    'NH2': 'N',
    'NO2': '[N+]([O-])=O',
    'OH': 'O',
    'OMe': 'OC',
    'Ph': 'C1=CC=CC=C1',
    'SCH3': 'SC',
    'SH': 'S',
    'SiH3': '[SiH3]',
    'Allyl': 'C=C',
    'Bn': 'CC1=CC=CC=C1',
    'Propyl': 'CCC',
    'tBu': 'C(C)(C)C',
    'TMS': '[Si](C)(C)C',
}

ligand_dict_R2 = {
    '26-F-Ph': 'C3(F)=CC=CC(F)=C3',
    '35-F-Ph': 'C2=C(F)C=C(F)C=C2',
    'Br': 'Br',
    'CF3': 'FC(F)(F)',
    'CH2Cl': 'ClC',
    'CH2F': 'FC',
    'CH2OH': 'OC',
    'CHO': 'O=C',
    'Cl': 'Cl',
    'COCH3': 'O=C(C)',
    'CONH2': 'O=C(N)',
    'COOCH3': 'O=C(OC)',
    'COOH': 'O=C(O)',
    'CN': 'N#C',
    'Cy': 'C1CCCCC1',
    'Et': 'CC',
    'F': 'F',
    'H': '',
    'I': 'I',
    'iPr': 'CC(C)',
    'Me': 'C',
    'Mes': 'CC1=CC(C)=CC(C)=C1',
    'NCH3_2': 'CN(C)',
    'NHCH3': 'CN',
    'NH2': 'N',
    'NO2': '[O-][N+](=O)',
    'OH': 'O',
    'OMe': 'CO',
    'Ph': 'C1=CC=CC=C1',
    'SCH3': 'CS',
    'SH': 'S',
    'SiH3': '[SiH3]',
    'Allyl': 'C=C',
    'Bn': 'C1=CC=CC=C1C',
    'Propyl': 'CCC',
    'tBu': 'CC(C)(C)',
    'TMS': 'C[Si](C)(C)',
}


SMILES = list()

for i in range(len(R1)):
    smi = '[R2]P([R3])[R1]'
    smi = smi.replace('[R1]', ligand_dict_R1[updated_df.loc[i].R1])
    smi = smi.replace('[R2]', ligand_dict_R2[updated_df.loc[i].R2])
    if updated_df.loc[i].R3 == 'H':
      smi = smi.replace('([R3])', '')
    else:
      smi = smi.replace('[R3]', ligand_dict_R1[updated_df.loc[i].R3])
    SMILES.append(smi)

len(SMILES)

updated_df['SMILES'] = SMILES
updated_df.head()

,R1,R2,R3,Filename,TEP,TEPcorr,SMILES
0,26-F-Ph,26-F-Ph,tBu,tBu-phos-_9=26-F-Ph_10=26-F-Ph.log,2167.36,2066.81,C3(F)=CC=CC(F)=C3P(C(C)(C)C)C3=C(F)C=CC=C3F
1,26-F-Ph,35-F-Ph,tBu,tBu-phos-_9=26-F-Ph_10=35-F-Ph.log,2165.69,2065.11,C2=C(F)C=C(F)C=C2P(C(C)(C)C)C3=C(F)C=CC=C3F
2,26-F-Ph,Br,tBu,tBu-phos-_9=26-F-Ph_10=Br.log,2175.11,2074.71,BrP(C(C)(C)C)C3=C(F)C=CC=C3F
3,26-F-Ph,CF3,tBu,tBu-phos-_9=26-F-Ph_10=CF3.log,2176.99,2076.63,FC(F)(F)P(C(C)(C)C)C3=C(F)C=CC=C3F
4,26-F-Ph,CH2Cl,tBu,tBu-phos-_9=26-F-Ph_10=CH2Cl.log,2169.93,2069.43,ClCP(C(C)(C)C)C3=C(F)C=CC=C3F


In [ ]:
updated_df.to_csv('UPDATED_TEPid_20230715.csv')

# RDkit

In [ ]:
duplicates_smiles = updated_df[updated_df['SMILES'].duplicated()]['SMILES'].values
print("Number of duplicate SMILES: ", len(duplicates_smiles))

Number of duplicate SMILES:  0


In [ ]:
# canonical_smiles(updated_df.SMILES)
Mol_descriptors, desc_names = RDkit_descriptors(updated_df.SMILES)
df_rdkit = pd.DataFrame(Mol_descriptors, columns=desc_names)
df_rdkit = df_rdkit.select_dtypes([np.number])
df_rdkit.head()

,MaxEStateIndex,MinEStateIndex,MaxAbsEStateIndex,MinAbsEStateIndex,qed,MolWt,HeavyAtomMolWt,ExactMolWt,NumValenceElectrons,NumRadicalElectrons,...,fr_sulfide,fr_sulfonamd,fr_sulfone,fr_term_acetylene,fr_tetrazole,fr_thiazole,fr_thiocyan,fr_thiophene,fr_unbrch_alkane,fr_urea
0,15.34,-4.57,15.34,1.42,0.57,314.26,299.14,314.08,112,0,...,0,0,0,0,0,0,0,0,0,0
1,15.37,-4.43,15.37,1.40,0.57,314.26,299.14,314.08,112,0,...,0,0,0,0,0,0,0,0,0,0
2,14.45,-3.72,14.45,1.14,0.68,281.08,268.98,279.98,78,0,...,0,0,0,0,0,0,0,0,0,0
3,14.51,-6.09,14.51,1.48,0.52,270.18,258.08,270.06,96,0,...,0,0,0,0,0,0,0,0,0,0
4,14.79,-4.16,14.79,1.28,0.55,250.66,236.54,250.05,84,0,...,0,0,0,0,0,0,0,0,0,0


# **Running ML**

In [ ]:
X_orig = df_rdkit.copy()


corr_matrix = X_orig.corr().abs()
mask = np.triu(np.ones_like(corr_matrix, dtype=bool))
tri_df = corr_matrix.mask(mask)

# # Find index of columns with correlation greater than 0.95
to_drop = [c for c in tri_df.columns if any(tri_df[c] > 0.95)]

print(to_drop)

NameError: ignored

In [ ]:
X = X_orig.drop(to_drop, axis=1)
y = updated_df['TEPcorr']
X.shape

NameError: ignored

In [ ]:
X.to_csv('X_full-FEATS.csv', index=None)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=12)
X_train_2, X_val, y_train_2, y_val = train_test_split(X_train, y_train, test_size=0.1, random_state=12)

In [ ]:
model = LGBMRegressor(random_state=12)

model.fit(X_train, y_train)

plotter(model, X_train, y_train, X_test, y_test)

In [ ]:
def objective(trial, X_train=X_train, y_train=y_train):

    train_x, test_x, train_y, test_y = train_test_split(X_train, y_train, test_size=0.1, random_state=42)
    param = {
        'metric': 'rmse',
        'random_state': 12,
        'n_estimators': 200,
        'reg_alpha': trial.suggest_loguniform('reg_alpha', 1e-3, 10.0),
        'reg_lambda': trial.suggest_loguniform('reg_lambda', 1e-3, 10.0),
        'colsample_bytree': trial.suggest_categorical('colsample_bytree', [0.3,0.4,0.5,0.6,0.7,0.8,0.9, 1.0]),
        'subsample': trial.suggest_categorical('subsample', [0.4,0.5,0.6,0.7,0.8,1.0]),
        'learning_rate': trial.suggest_categorical('learning_rate', [0.006,0.008,0.01,0.014,0.017,0.02]),
        'max_depth': trial.suggest_categorical('max_depth', [10,20,100]),
        'num_leaves' : trial.suggest_int('num_leaves', 2, 1000),
        'min_child_samples': trial.suggest_int('min_child_samples', 1, 300),
        'cat_smooth' : trial.suggest_int('cat_smooth', 1, 100),
        'max_bin' : trial.suggest_int('max_bin', 2, 255)
    }
    model = LGBMRegressor(**param)

    model.fit(train_x,train_y,eval_set=[(test_x,test_y)],early_stopping_rounds=100,verbose=False)

    preds = model.predict(test_x)

    rmse = mean_squared_error(test_y, preds,squared=False)

    return rmse

study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=50)
print('Number of finished trials:', len(study.trials))
print('Best trial:', study.best_trial.params)

In [ ]:
best_params = study.best_params

print(best_params)

In [ ]:
model = LGBMRegressor(random_state=12, n_estimators=200, **best_params)

model.fit(X_train, y_train)

plotter(model, X_train, y_train, X_test, y_test)

In [ ]:
model = LGBMRegressor(random_state=12, n_estimators=200, **best_params)

test_learner(X_train, y_train, model)

In [ ]:
model.fit(X_train, y_train)

#calculate permutation importance for testing data
result_test = permutation_importance(
    model, X_test, y_test, n_repeats=10, random_state=seed, n_jobs=-1, scoring='neg_root_mean_squared_error'
    )

sorted_importances_idx_test = result_test.importances_mean.argsort()
importances_test = pd.DataFrame(
    result_test.importances[sorted_importances_idx_test[-20:]].T,
    columns=X.columns[sorted_importances_idx_test[-20:]],
)

#calculate permutation importance for training data
result_train = permutation_importance(
    model, X_train, y_train, n_repeats=10, random_state=seed, n_jobs=-1, scoring='neg_root_mean_squared_error'
)

sorted_importances_idx_train = result_train.importances_mean.argsort()
importances_train = pd.DataFrame(
    result_train.importances[sorted_importances_idx_train[-20:]].T,
    columns=X.columns[sorted_importances_idx_train[-20:]],
)

f, axs = plt.subplots(1,2,figsize=(15,5))

importances_test.plot.box(vert=False, whis=10, ax = axs[0])
axs[0].set_title("Permutation Importances (test set)")
axs[0].axvline(x=0, color="k", linestyle="--")
axs[0].set_xlabel("Increase in RMSE (eV)")
axs[0].figure.tight_layout()

importances_train.plot.box(vert=False, whis=10, ax = axs[1])
axs[1].set_title("Permutation Importances (train set)")
axs[1].axvline(x=0, color="k", linestyle="--")
axs[1].set_xlabel("Increase in RMSE (eV)")
axs[1].figure.tight_layout()

In [ ]:
X_reduced = pd.DataFrame()
X_reduced['BCUT2D_MWHI'] = X['VSA_EState3']
X_reduced['EState_VSA9'] = X['EState_VSA9']
X_reduced['BCUT2D_CHGLO'] = X['BCUT2D_CHGLO']
X_reduced['VSA_EState1'] = X['VSA_EState1']
X_reduced['Ipc'] = X['Ipc']
X_reduced['BCUT2D_MRHI'] = X['BCUT2D_MRHI']
X_reduced['SlogP_VSA12'] = X['SlogP_VSA12']
X_reduced['EState_VSA1'] = X['EState_VSA1']
X_reduced['VSA_EState3'] = X['VSA_EState3']
X_reduced['VSA_EState10'] = X['VSA_EState10']
X_reduced['fr_halogen'] = X['fr_halogen']
X_reduced['EState_VSA10'] = X['EState_VSA10']
X_reduced['BCUT2D_LOGPHI'] = X['BCUT2D_LOGPHI']
X_reduced['VSA_EState8'] = X['VSA_EState8']
X_reduced['BCUT2D_CHGHI'] = X['BCUT2D_CHGHI']
X_reduced['MolMR'] = X['MolMR']
X_reduced['MaxPartialCharge'] = X['MaxPartialCharge']
X_reduced['Kappa1'] = X['Kappa1']
X_reduced['NumRotatableBonds'] = X['NumRotatableBonds']
X_reduced['SlogP_VSA6'] = X['SlogP_VSA6']

In [ ]:
X_reduced.to_csv('X_reduced-FEATS.csv', index=None)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_reduced, y, test_size=0.1, random_state=seed)
X_train_2, X_val, y_train_2, y_tval = train_test_split(X_train, y_train, test_size=0.1, random_state=seed)

In [ ]:
def objective(trial, X_train=X_train, y_train=y_train):

    train_x, test_x, train_y, test_y = train_test_split(X_train, y_train, test_size=0.1, random_state=42)
    param = {
        'metric': 'rmse',
        'random_state': 12,
        'n_estimators': 200,
        'reg_alpha': trial.suggest_loguniform('reg_alpha', 1e-3, 10.0),
        'reg_lambda': trial.suggest_loguniform('reg_lambda', 1e-3, 10.0),
        'colsample_bytree': trial.suggest_categorical('colsample_bytree', [0.3,0.4,0.5,0.6,0.7,0.8,0.9, 1.0]),
        'subsample': trial.suggest_categorical('subsample', [0.4,0.5,0.6,0.7,0.8,1.0]),
        'learning_rate': trial.suggest_categorical('learning_rate', [0.006,0.008,0.01,0.014,0.017,0.02]),
        'max_depth': trial.suggest_categorical('max_depth', [10,20,100]),
        'num_leaves' : trial.suggest_int('num_leaves', 2, 1000),
        'min_child_samples': trial.suggest_int('min_child_samples', 1, 300),
        'cat_smooth' : trial.suggest_int('cat_smooth', 1, 100),
        'max_bin' : trial.suggest_int('max_bin', 2, 255)
    }
    model = LGBMRegressor(**param)

    model.fit(train_x,train_y,eval_set=[(test_x,test_y)],early_stopping_rounds=100,verbose=False)

    preds = model.predict(test_x)

    rmse = mean_squared_error(test_y, preds,squared=False)

    return rmse

study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=50)
print('Number of finished trials:', len(study.trials))
print('Best trial:', study.best_trial.params)

In [ ]:
best_params = study.best_params

print(best_params)

In [ ]:
model = LGBMRegressor(random_state=12, n_estimators=500, **best_params)

model.fit(X_train, y_train)

plotter(model, X_train, y_train, X_test, y_test)

In [ ]:
model = LGBMRegressor(random_state=12, n_estimators=500, **best_params)

test_learner(X_train, y_train, model)

In [ ]:
X_reduced.to_csv('X_reduced.csv')

NameError: ignored

In [ ]:
shap.initjs()
explainer = shap.TreeExplainer(model)
shap_values = explainer.shap_values(X_train)

shap.summary_plot(shap_values, features=X_train, feature_names=X.columns)

In [ ]:
from sklearn.model_selection import learning_curve

train_sizes, train_scores, test_scores = learning_curve(model, X_reduced, y, cv=5, scoring='neg_root_mean_squared_error', n_jobs=-1,
                                                        train_sizes=np.linspace(0.01, 1.0, 50))

train_mean = -1 * np.mean(train_scores, axis=1)
train_std = -1 * np.std(train_scores, axis=1)

test_mean = -1 * np.mean(test_scores, axis=1)
test_std = -1 * np.std(test_scores, axis=1)

plt.subplots(1, figsize=(10,10))
plt.plot(train_sizes, train_mean, '--', color="#111111",  label="Training score")
plt.plot(train_sizes, test_mean, color="#111111", label="Cross-validation score")

plt.fill_between(train_sizes, train_mean - train_std, train_mean + train_std, color="#DDDDDD")
plt.fill_between(train_sizes, test_mean - test_std, test_mean + test_std, color="#DDDDDD")

plt.xlabel("Training Set Size"), plt.ylabel("RMSE (cm$^{-1}$)"), plt.legend(loc="best")
plt.tight_layout()
plt.show()

In [ ]:
import sklearn_json as skljson
from sklearn.ensemble import GradientBoostingRegressor

model = RandomForestRegressor(max_depth=None, max_features='sqrt', n_estimators=500).fit(X_train, y_train)

#skljson.to_json(model, "RandomForest_TEPid.json")

In [ ]:
import pickle
with open('LGBMReg_20230715.pkl', 'wb') as f:
  pickle.dump(model,f)

In [ ]:
with open('GBReg_20230303.pkl', 'rb') as f:
    clf2 = pickle.load(f)

In [ ]:
model.get_params()

In [ ]:
from sklearn.tree import DecisionTreeRegressor

def serialize_tree(tree):
    serialized_tree = tree.__getstate__()
    # serialized_tree['nodes_dtype'] = serialized_tree['nodes'].dtype
    dtypes = serialized_tree['nodes'].dtype
    serialized_tree['nodes'] = serialized_tree['nodes'].tolist()
    serialized_tree['values'] = serialized_tree['values'].tolist()

    return serialized_tree, dtypes

def serialize_decision_tree_regressor(model):
    tree, dtypes = serialize_tree(model.tree_)
    serialized_model = {
        'meta': 'decision-tree-regression',
        'feature_importances_': model.feature_importances_.tolist(),
        'max_features_': model.max_features_,
        'n_features_': model.n_features_,
        'n_outputs_': model.n_outputs_,
        'tree_': tree
    }

    # serialized_model.

    tree_dtypes = []
    for i in range(0, len(dtypes)):
        tree_dtypes.append(dtypes[i].str)

    serialized_model['tree_']['nodes_dtype'] = tree_dtypes

    return serialized_model

In [ ]:
def serialize_random_forest_regressor(model):

    serialized_model = {
        'meta': 'rf-regression',
        'max_depth': model.max_depth,
        'min_samples_split': model.min_samples_split,
        'min_samples_leaf': model.min_samples_leaf,
        'min_weight_fraction_leaf': model.min_weight_fraction_leaf,
        'max_features': model.max_features,
        'max_leaf_nodes': model.max_leaf_nodes,
        'min_impurity_decrease': model.min_impurity_decrease,
        'n_features_': model.n_features_,
        'n_outputs_': model.n_outputs_,
        'estimators_': [serialize_decision_tree_regressor(decision_tree) for decision_tree in model.estimators_],
        'params': model.get_params()
    }

    if 'oob_score_' in model.__dict__:
        serialized_model['oob_score_'] = model.oob_score_
    if 'oob_decision_function_' in model.__dict__:
        serialized_model['oob_prediction_'] = model.oob_prediction_.tolist()

    return serialized_model

In [ ]:
serialize_random_forest_regressor(model)

In [ ]:
import json

def to_json(model, model_name):
  with open(model_name, 'w') as model_json:
    json.dump(serialize_random_forest_regressor(model), model_json)

to_json(model, "RandomForest_TEPid.json")